# Image inspection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from aicsimageio import AICSImage

from scip_workflows.common import *


In [ ]:
try:
    input_file = snakemake.input[0]
    output = snakemake.output[0]
except NameError:
    input_file = Path(
        "/home/maximl/scratch/data/vsc/datasets/cd7/800/Experiment-800.czi"
    )
    output = Path("/home/maximl/scratch/data/vsc/datasets/cd7/800/scenes.txt")


In [ ]:
im = AICSImage(input_file, reconstruct_mosaic=False)


In [ ]:
pattern = r"P(?P<position>[0-9]+)-(?P<exp>[A-Z])(?P<replicate>[0-9])"
df = pandas.DataFrame(map(lambda a: re.match(pattern, a).groupdict(), im.scenes))
df = df[df["exp"] == "D"]
df["position"] = df["position"].astype(int)
df["replicate"] = df["replicate"].astype(int)
df = df.set_index(["replicate", "position"])


In [ ]:
im.set_scene("P1-D1")


In [ ]:
fig, ax = plt.subplots(dpi=150)
ax.imshow(
    numpy.max(im.get_image_data("ZXY", C=5), axis=0), origin="lower", cmap="Greys"
)
ax.set_axis_off()


In [ ]:
def draw_tile(data, channel, color=None, hue=None):
    p, rep = data.position.iloc[0], data.replicate.iloc[0]
    im.set_scene(f"P{p}-D{rep}")

    ax = plt.gca()

    ax.imshow(
        numpy.max(im.get_image_data("ZXY", C=channel), axis=0),
        origin="lower",
        cmap="viridis",
    )
    ax.set_axis_off()
    print(f"{p}-{rep}", end=" ")


In [ ]:
# DAPI

grid = seaborn.FacetGrid(
    data=df.reset_index(), col="position", row="replicate", margin_titles=True
)
grid.map_dataframe(draw_tile, channel=0)
for ax in grid.axes.ravel():
    ax.set_axis_off()


In [ ]:
bad_indices = [(2, 1), (2, 15), (2, 21), (3, 25), (4, 21), (4, 25), (5, 25), (5, 21)]

df = df.drop(index=bad_indices)


In [ ]:
# CD45 - EGFP

grid = seaborn.FacetGrid(
    data=df.reset_index(), col="position", row="replicate", margin_titles=True
)
grid.map_dataframe(draw_tile, channel=1)
for ax in grid.axes.ravel():
    ax.set_axis_off()


In [ ]:
bad_indices = [(2, 8)]

df = df.drop(index=bad_indices)


In [ ]:
# siglec8 - RPe

grid = seaborn.FacetGrid(
    data=df.reset_index(), col="position", row="replicate", margin_titles=True
)
grid.map_dataframe(draw_tile, channel=2)
for ax in grid.axes.ravel():
    ax.set_axis_off()


In [ ]:
bad_indices = [(2, 3), (2, 23), (5, 5)]

df = df.drop(index=bad_indices)


In [ ]:
# CD15 - APC

grid = seaborn.FacetGrid(
    data=df.reset_index(), col="position", row="replicate", margin_titles=True
)
grid.map_dataframe(draw_tile, channel=3)
for ax in grid.axes.ravel():
    ax.set_axis_off()


In [ ]:
with open(output, "w") as fh:
    fh.write("    scenes:\n")
    for s in (
        df.reset_index()
        .apply(lambda r: f"P{r.position}-{r.exp}{r.replicate}", axis=1)
        .values.tolist()
    ):
        fh.write(f"      - {s}\n")
